In [1]:
import plotly
import pandas
import numpy
import scipy
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.graph_objs.scatter import Marker

In [2]:
# Fitness máximo possível para cada instância do problema
MAX_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 21,
    "abba": 22,
    "aman-aplan": 19,
    "prime": 20,
    "four": 21,
    "order": 21,
    "triples": 21,
    "glob": 21,
    "balance": 32,
    "powers": 11,
    "long-count": 1,
    "alphabetical": 17,
}

# Fitness que o Bartoli conseguiu segundo expressão regular gerada
BARTOLI_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 18,
    "abba": 20,
    "aman-aplan": 16,
    "prime": 13,
    "four": 21,
    "order": 21,
    "triples": 15,
    "glob": 19,
    "balance": 29,
    "powers": 7,
    "long-count": 1,
    "alphabetical": 12,
}

NORVIG_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 21,
    "abba": 21,
    "aman-aplan": 19,
    "prime": 19,
    "four": 21,
    "order": 21,
    "triples": 21,
    "glob": 0,
    "balance": 31,
    "powers": 11,
    "long-count": 1,
    "alphabetical": 17,
}

FITNESS_LINE = go.Scatter(
    name="Pontuação Máxima",
    x=list(MAX_FITNESS.keys()),
    y=list(MAX_FITNESS.values()),
    mode='lines',
)

BARTOLI_LINE = go.Scatter(
    name="Algoritmo Genético",
    x=list(BARTOLI_FITNESS.keys()),
    y=list(BARTOLI_FITNESS.values()),
    mode='markers',
)

NORVIG_LINE = go.Scatter(
    name="Algoritmo Exato (Norvig)",
    x=list(NORVIG_FITNESS.keys()),
    y=list(NORVIG_FITNESS.values()),
    mode='markers',
)


COLOR_GREEN = "#00b100"
COLOR_RED   = "#cc0000"


In [3]:
# Importando os dados
ILS_S_09 = pandas.read_csv("../09/ILS_Shrink.csv") #.sort_values(by=["Nome"])
ILS_S_10 = pandas.read_csv("../10/ILS_Shrink.csv") #.sort_values(by=["Nome"])

ILS_S_300k = pandas.read_csv("./ILS_Shrink_300k.csv") #.sort_values(by=['Nome'])
ILS_S_600k = pandas.read_csv("./ILS_Shrink_600k.csv") #.sort_values(by=['Nome'])

ILS_S_09.Tempo_total = ILS_S_09.Tempo_total.map(lambda x: x // 1000)
ILS_S_10.Tempo_total = ILS_S_10.Tempo_total.map(lambda x: x // 1000)
ILS_S_300k.Tempo_total = ILS_S_300k.Tempo_total.map(lambda x: x // 1000)
ILS_S_600k.Tempo_total = ILS_S_600k.Tempo_total.map(lambda x: x // 1000)

ILS_S_300k.Tempo_para_encontrar_melhor_solucao = ILS_S_300k.Tempo_para_encontrar_melhor_solucao.map(lambda x: x // 1000)
ILS_S_600k.Tempo_para_encontrar_melhor_solucao = ILS_S_600k.Tempo_para_encontrar_melhor_solucao.map(lambda x: x // 1000)


In [4]:
# Número de timeouts

def get_timeout_graph(title, d):
    data = []
    nomes = pandas.unique(d.Nome)
    timedout = []
    not_timedout = []
    
    for nome in pandas.unique(d.Nome):
        i = len(d[(d.Nome == nome) & (d.Timed_out == 0)].Nome)
        not_timedout.append(i)

        i = len(d[(d.Nome == nome) & (d.Timed_out == 1)].Nome)
        timedout.append(i)
        
    trace1 = go.Bar(
        name='ok', 
        x=nomes,
        y=not_timedout,
        marker=dict(color=COLOR_GREEN),
        textposition='auto',
        text=not_timedout)
    
    trace2 = go.Bar(
        name='Timedout',
        x=nomes,
        y=timedout,
        marker=dict(color=COLOR_RED),
        textposition='auto',
        text=timedout)
    
    data = [trace1, trace2]
    layout = go.Layout(title=title, barmode='stack')
    fig = go.Figure(data=data, layout=layout)
    return fig


In [5]:
# py.iplot(get_timeout_graph('ok / Timedout 300k', ILS_S_300k), filename='stacked-bar')

In [6]:
py.iplot(get_timeout_graph('ok / Timedout 600k', ILS_S_600k), filename='stacked-bar')

In [7]:
# Boxplot comparando 'Tempo total de execução' entre os ILS_S
ONE_MINUTE_LINE = go.Scatter(name="1 minute",
                             x=ILS_S_300k.Nome,
                             y=[60 for i in ILS_S_300k.Nome],
                             mode='lines')

TWO_MINUTES_LINE = go.Scatter(name="2 minutes",
                              x=ILS_S_300k.Nome,
                              y=[120 for i in ILS_S_300k.Nome],
                              mode='lines')

THREE_MINUTES_LINE = go.Scatter(name="3 minutes",
                                x=ILS_S_300k.Nome,
                                y=[180 for i in ILS_S_300k.Nome],
                                mode='lines')

FOUR_MINUTES_LINE = go.Scatter(name="4 minutes",
                               x=ILS_S_300k.Nome,
                               y=[240 for i in ILS_S_300k.Nome],
                               mode='lines')

FIVE_MINUTES_LINE = go.Scatter(name="5 minutes",
                               x=ILS_S_300k.Nome,
                               y=[300 for i in ILS_S_300k.Nome],
                               mode='lines')

data = [
    ONE_MINUTE_LINE,
    TWO_MINUTES_LINE,
    THREE_MINUTES_LINE,
    FOUR_MINUTES_LINE,
    FIVE_MINUTES_LINE,
#     go.Box(name="ILS_S anterior", x=ILS_S_10.Nome, y=ILS_S_10.Tempo_total),
    go.Box(name="ILS_S 300k", x=ILS_S_300k.Nome, y=ILS_S_300k.Tempo_total, marker=dict(color=COLOR_GREEN)),
    go.Box(name="ILS_S 600k", x=ILS_S_600k.Nome, y=ILS_S_600k.Tempo_total, marker=dict(color=COLOR_RED)),
]

layout = go.Layout(title="Tempo total de execução em seconds (com Timedout)",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='fitness'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [8]:
data = [
    ONE_MINUTE_LINE,
    go.Box(name="ILS_S 300k", x=ILS_S_300k[(ILS_S_300k.Timed_out == 0)].Nome, y=ILS_S_300k[(ILS_S_300k.Timed_out == 0)].Tempo_total, marker=dict(color=COLOR_GREEN)),
    go.Box(name="ILS_S 600k", x=ILS_S_600k[(ILS_S_600k.Timed_out == 0)].Nome, y=ILS_S_600k[(ILS_S_600k.Timed_out == 0)].Tempo_total, marker=dict(color=COLOR_RED)),
]

layout = go.Layout(title="Tempo total de execução em seconds (sem Timedout)",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='fitness'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [9]:
# Médias de número de comparações para encontrar melhor fitness


MAX_LINE_1 = go.Scatter(name="300k line", x=ILS_S_10.Nome.unique(), y=[300000 for i in range(15)], mode='lines', marker=Marker(color='rgb(102, 51, 153)'))
MAX_LINE_2 = go.Scatter(name="600k line", x=ILS_S_10.Nome.unique(), y=[600000 for i in range(15)], mode='lines', marker=Marker(color='rgb(255, 77, 77)'))


data = [
    go.Box(name="ILS_S 300k", x=ILS_S_300k.Nome, y=ILS_S_300k.Numero_de_comparacoes, marker=dict(color=COLOR_GREEN)),
    go.Box(name="ILS_S 600k", x=ILS_S_600k.Nome, y=ILS_S_600k.Numero_de_comparacoes, marker=dict(color=COLOR_RED)),
    
    MAX_LINE_1,
    MAX_LINE_2,
]

layout = go.Layout(title="Número de comparações até encontrar Melhor Fitness",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='Quantidade de comprações'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [10]:
data = [
    ONE_MINUTE_LINE,
    TWO_MINUTES_LINE,
    go.Box(name="300k", x=ILS_S_300k.Nome, y=ILS_S_300k.Tempo_para_encontrar_melhor_solucao, marker=dict(color=COLOR_GREEN)),
    go.Box(name="600k", x=ILS_S_600k.Nome, y=ILS_S_600k.Tempo_para_encontrar_melhor_solucao, marker=dict(color=COLOR_RED)),
]

layout = go.Layout(title="Tempo até encontrar a melhor solução (em segundos)",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='Tempo até melhor solução'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [11]:
data = [
    ONE_MINUTE_LINE,
    go.Box(name="300k", x=ILS_S_300k[(ILS_S_300k.Timed_out == 0)].Nome, y=ILS_S_300k[(ILS_S_300k.Timed_out == 0)].Tempo_para_encontrar_melhor_solucao, marker=dict(color=COLOR_GREEN)),
    go.Box(name="600k", x=ILS_S_600k[(ILS_S_600k.Timed_out == 0)].Nome, y=ILS_S_600k[(ILS_S_600k.Timed_out == 0)].Tempo_para_encontrar_melhor_solucao, marker=dict(color=COLOR_RED)),
]

layout = go.Layout(title="Tempo até encontrar a melhor solução sem Timedout (em segundos)",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='Tempo até melhor solução'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [12]:
# Boxplot comparando 'melhor fitness'
data = [
    FITNESS_LINE,
    go.Box(name="300k", x=ILS_S_300k.Nome, y=ILS_S_300k.Melhor_fitness, marker=dict(color=COLOR_GREEN)),
    go.Box(name="600k", x=ILS_S_600k.Nome, y=ILS_S_600k.Melhor_fitness, marker=dict(color=COLOR_RED)),
]

layout = go.Layout(title="Melhor fitness",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='fitness'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)
